# 3、Adagrad

Adagrad优化算法 被称为<mark>自适应学习率优化算法</mark>，

之前我们讲的随机梯度下降法，对所有的参数，都是使用相同的、固定的学习率进行优化的，但是不同的参数的梯度差异可能很大，使用相同的学习率，效果不会很好

<br>

> <font color="green">| 举例：</font>
>
> 假设损失函数是 $f(x) = x_1^2 + 10x_2^2$， $x$ 和 $y$ 的初值分别为 $x_1=40, x_2=20$
>
> (通过观察，我们即可知道， $x_1=0, x_2=0$ 就是两个参数的极值点)
>
> $\rightarrow \frac{\partial loss}{\partial x_1} = 80$, $\frac{\partial loss}{\partial x_2} = 400$    $\rightarrow x_1$将要移动的幅度 小于$x_2$将要移动的幅度
>
> 而 $x_1$ 距离离极值点 $x_1=0$ 是较远的，所以，我们使用梯度下降法，效果并不会好

<br>

<font color="brown">Adagrad 思想：对于不同参数，设置不同的学习率</font>

方法：对于每个参数，初始化一个 累计平方梯度 $r=0$，然后每次将该参数的梯度平方求和累加到这个变量 $r$ 上：
$$ r \leftarrow r + g^2 $$

然后，在更新这个参数的时候，学习率就变为：
$$ \frac{\eta}{\sqrt{r+\delta}} $$

权重更新：
$$ w \leftarrow w - \frac{\eta}{\sqrt{r+\delta}} * g $$

其中， $g$为梯度； $r$为累积平方梯度(初始为0)； $\eta$为学习率； $\delta$为小参数，避免分母为0，一般取值为$10^{-10}$

这样，不同的参数由于梯度不同，他们对应的$r$大小也就不同，所以学习率也就不同，这就实现了自适应的学习率。

总结： Adagrad 的核心想法就是，如果一个参数的梯度一直都非常大，那么其对应的学习率就变小一点，防止震荡，而一个参数的梯度一直都非常小，那么这个参数的学习率就变大一点，使得其能够更快地更新，这就是Adagrad算法加快深层神经网络的训练速度的核心。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# ---------------------------
# 设置Matplotlib支持中文
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 
# ---------------------------

def loss_function(x, y):
    # f(x) = x_1^2 + 10x_2^2
    # 使用Adagrad文档中的例子，展示梯度差异大时的效果
    return x**2 + 10 * y**2

# 初始化参数
# 文档中的例子: x1=40, x2=20
x_start, y_start = 40.0, 20.0
eta = 1.5 # Adagrad通常需要较大的初始学习率
num_epochs = 50
epsilon = 1e-10

# 1. 普通梯度下降 (无Adagrad) - 用于对比
x, y = x_start, y_start
gd_lr = 0.05 # 普通GD学习率必须取小，以适应陡峭的y方向(梯度大)，否则发散
result_gd = [(x,y)]

for i in range(num_epochs):
    grads_x = 2 * x
    grads_y = 20 * y
    
    x -= gd_lr * grads_x
    y -= gd_lr * grads_y
    result_gd.append((x,y))

# 2. Adagrad算法
x_ada, y_ada = x_start, y_start
r_x, r_y = 0, 0 # 累积平方梯度 (Accumulated Squared Gradients)
result_ada = [(x_ada, y_ada)]

for i in range(num_epochs):
    # 计算梯度
    g_x = 2 * x_ada
    g_y = 20 * y_ada
    
    # 累积平方梯度 r <- r + g^2
    r_x += g_x ** 2
    r_y += g_y ** 2
    
    # 参数更新 w <- w - eta / sqrt(r + epsilon) * g
    x_ada -= (eta / np.sqrt(r_x + epsilon)) * g_x
    y_ada -= (eta / np.sqrt(r_y + epsilon)) * g_y
    
    result_ada.append((x_ada, y_ada))

# --- 2D 绘图代码 ---

# 提取轨迹
x_traj_gd = [p[0] for p in result_gd]
y_traj_gd = [p[1] for p in result_gd]

x_traj_ada = [p[0] for p in result_ada]
y_traj_ada = [p[1] for p in result_ada]

# 创建网格
X_range = np.arange(-50, 50, 0.5)
Y_range = np.arange(-30, 30, 0.5)
X, Y = np.meshgrid(X_range, Y_range)
Z = loss_function(X, Y)

plt.figure(figsize=(10, 6))

# 绘制等高线
contour = plt.contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.6)
plt.clabel(contour, inline=True, fontsize=8)

# 绘制轨迹
plt.plot(x_traj_gd, y_traj_gd, 'r--o', label=f'GD (lr={gd_lr})', alpha=0.5, markersize=4)
plt.plot(x_traj_ada, y_traj_ada, 'b-d', label=f'Adagrad (lr={eta})', linewidth=2)

# 起点终点
plt.plot(x_start, y_start, 'go', markersize=10, label='Start')
plt.plot(0, 0, 'k*', markersize=15, label='Optimal (0,0)')

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title(f'Adagrad vs GD 优化轨迹对比\n函数: $f(x)=x_1^2 + 10x_2^2$', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# --- 3D 绘图代码 ---

# 计算轨迹上的Z值
z_traj_gd = [loss_function(p[0], p[1]) for p in result_gd]
z_traj_ada = [loss_function(p[0], p[1]) for p in result_ada]

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制表面
surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.4, edgecolor='none')

# 绘制轨迹
ax.plot(x_traj_gd, y_traj_gd, z_traj_gd, 'r--', linewidth=1, label='GD Path')
ax.plot(x_traj_ada, y_traj_ada, z_traj_ada, 'b-', linewidth=3, label='Adagrad Path')

# 标记
ax.scatter([x_start], [y_start], [loss_function(x_start, y_start)], c='g', s=60, label='Start', zorder=20)
ax.scatter([0], [0], [0], c='k', marker='*', s=150, label='Optimal', zorder=20)

ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_zlabel('Loss')
ax.set_title('3D View: Adagrad能够更好地适应不同方向的梯度', fontsize=14)
ax.view_init(elev=30, azim=120) 

fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, label='Loss')
plt.legend()
plt.show()